In [4]:
import pandas as pd
import numpy as np
import math
from qiskit import QuantumCircuit
from qiskit import Aer, execute
import os

ModuleNotFoundError: No module named 'qiskit'

### Lectura de archivos de datos
1. Se importa como **'data'** el archivo de datos proporcionado. Los datos contienen los pedidos Train y Test a comparar. Se compararan todos los pedidos Train con los Test.
   - Se descartan las columnas que no se utilizaran en la comparación.

&nbsp;
2. Se importa como **'data denominacion'** el archivo que contiene el nombre real de las piezas codificadas.

In [ ]:
#data = pd.read_csv('Montabilidad_to_Quantum.csv',delimiter=';', encoding='latin-1')
data = pd.read_csv('Dataset_Quantum.csv',delimiter=',', encoding='latin-1')

#columnas_a_quitar = ['file_type','metadata_path', 'metadata_date', 'metadata_size', 'query_key', 'text', 'Expediente', 'Código', 'N_Produccion', 'Fecha_Tecnica', 'metadata_file']
columnas_a_quitar = ['Pedidos','Texto', 'Expediente', 'Codigo', 'N_Produccion']
#data = data.drop(columnas_a_quitar, axis=1)
data = data.drop(columnas_a_quitar, axis=1)

data_denominacion = pd.read_excel('Codigos Denominacion 447.xlsx')

Se filtran en dos DataFrame diferentes los vectores Test y Train

In [ ]:
def filtrado_vectores_train_test(data):
    vectores_train = data[data['Set'] == 'Train']
    vectores_test = data[data['Set'] == 'Test']
    return (vectores_test, vectores_train)

Función para normalizar un vector:  módulo = 1

In [ ]:
def normalizar_vector(vector):
    norma = np.linalg.norm(vector)
    return vector/norma

Se le llama padding a la operación de rellenar un vector con 0s hasta llegar a un tamanño deseado.

En este caso necesitamos rellenar los vectores de tal forma que correspondan a al tamaño $2^n$ para poder codificarlos en estados cuánticos. Siendo n el numerpo de qbits utilizados.

In [ ]:
def padding(vec, qubits):
    num_componentes_relleno = 2**qubits - len(vec)
    return np.pad(vec, (0, num_componentes_relleno), mode='constant', constant_values = 0)

Función que calcula el producto interno de dos estados cuánticos. 
> **input:** 2 vectores normalizados.

> **output:** producto interno. 0 si los vectores son totalmente diferentes 1 si son iguales.
1. Se comprueba que los vectores tengan el mismo tamaño, se calcula el numero de qbits necesarios para la operación y se aplica el padding a los vectores.
2. Se inicializan los estados cuánticos mediante la función initialize de Qiskit donde  los datos de los vectores se los codifica en amplitudes.
3. Se realizan las mediciones sobre la base canónica mediante la matriz Z de Pauli. Se procesa el resultado para obtener el producto interno.

La ejecución se hace sobre el simulador 'qasm_simulator'


In [ ]:
def inner_prod(vec1, vec2):

    if len(vec1) != len(vec2):
        raise ValueError('Lengths of states are not equal')

    numero_componentes = len(vec1)
    nqubits = math.ceil(np.log2(numero_componentes))

    vec1 = padding(vec1, nqubits)
    vec2 = padding(vec2, nqubits)

    circ = QuantumCircuit(nqubits+1,1)
    vec = np.concatenate([vec1,vec2])/np.sqrt(2)

    circ.initialize(vec, range(nqubits+1))
    circ.h(nqubits)
    circ.measure(nqubits,0)

    backend = Aer.get_backend('qasm_simulator')
    job = execute(circ, backend, shots=20000)

    result = job.result()
    outputstate = result.get_counts(circ)

    if ('0' in outputstate.keys()):
        m_sum = float(outputstate["0"])/20000
    else:
        m_sum = 0

    inner_product = 2*m_sum-1
    return inner_product

In [ ]:
def correlacion_vectores(vector_1, vector_2):
    test_normalizado = normalizar_vector(vector_1.iloc[ 1:-5].to_numpy())
    train_normalizado = normalizar_vector(vector_2.iloc[ 1:-5].to_numpy())
    return inner_prod(test_normalizado, train_normalizado)

In [ ]:
def itera_por_matriz_cuantica(vectores_test, vectores_train):
  for index_test, test in vectores_test.iterrows():
    for index_train, train in vectores_train.iterrows():
      correlacion_entre_variables = correlacion_vectores(test, train)
      yield [f'test_{index_test}', f'train_{index_train}', correlacion_entre_variables, test['Categoria'], test['N_Pedido'],train['N_Pedido']]

Funciones para obtener la difernecia de componentes entre el vector Test exótico y el Train similar

In [ ]:
def diferencias_componentes(exotico, similar):
    exotico = exotico.apply(lambda x: 1 if x != 0 else 0)
    similar = similar.apply(lambda x: 1 if x != 0 else 0)

    # Obtener las columnas que son 1 en `exotico` y 0 en `similar`
    condicion1 = (exotico == 1) & (similar == 0)
    diferencias_exotico = exotico[condicion1].index.tolist()

    # Obtener las columnas que son 1 en `similar` y 0 en `exotico`
    condicion2 = (similar == 1) & (exotico == 0)
    diferencias_similar = similar[condicion2].index.tolist()
    
    return diferencias_exotico,diferencias_similar

In [ ]:
def obtener_denominacion(comp, data_denominacion):
    # Verifica si hay coincidencias
    if comp in data_denominacion['Codigo'].values:
        denominacion = data_denominacion[data_denominacion['Codigo'] == comp]['Denominación'].values[0]
    else:
        denominacion = 'No ha habido coincidencias'
    return denominacion

Función de creación del la tabla final

In [ ]:
def creacion_resultado_final( exotico, similar, diferencia_exotico, diferencia_similar):
    # Crear el DataFrame
    columnas = [
        'Exotico',
        'Tipo_Vehículo',
        'Similar',
        'Descripción',
        'Componentes',
        'Id',
        'Categoria_exótico',
        'Categoria_similar',
        'Denominación'
    ]

    data = []

    # Iterar sobre la lista de diferencia exotico
    for comp in diferencia_exotico:
        entrada = [
            exotico['N_Pedido'],
            exotico['Tipo_vehiculo'],
            similar['N_Pedido'],
            'Componentes en el exotico que no aparecen en el similar',
            'Mas',
            comp,
            exotico['Categoria'],
            similar['Categoria'],
            obtener_denominacion(comp, data_denominacion)
        ]
        data.append(entrada)

    # Iterar sobre la lista de diferencia similar
    for comp in diferencia_similar:
        entrada = [
            exotico['N_Pedido'],
            exotico['Tipo_vehiculo'],
            similar['N_Pedido'],
            'Componentes en el similar que no aparecen en el exotico',
            'Menos',
            comp,
            exotico['Categoria'],
            similar['Categoria'],
            obtener_denominacion(comp, data_denominacion)
        ]
        data.append(entrada)

    # Crear el DataFrame final
    return pd.DataFrame(data, columns=columnas)

In [ ]:
def resultados_csv(ruta_guardado, resultado, nombre_resultado):
    # Comprobar si la ruta de guardado existe, si no, crearla
    if not os.path.exists(ruta_guardado):
        os.makedirs(ruta_guardado)

    # Unir la ruta de guardado con el nombre del archivo y la extensión .csv
    ruta_completa = os.path.join(ruta_guardado, f"{nombre_resultado}.csv")

    # Guardar el DataFrame en el archivo CSV
    resultado.to_csv(ruta_completa, index=False)


In [ ]:
columnas = ['Nombre_Test', 'Nombre_Train', 'Correlacion', 'Categoria', 'N_Pedido_test', 'N_Pedido_train']
categorias = data.groupby('Categoria')
resultados_categorias = pd.DataFrame()

for nom_categoria, categoria in categorias:
    if nom_categoria != 'CLASE V':
         vectores_test, vectores_train = filtrado_vectores_train_test(categoria) 
         
         resultados_correlacion = pd.DataFrame(itera_por_matriz_cuantica(vectores_test, vectores_train), columns=columnas)
         resultados_correlacion_ordenado = resultados_correlacion.sort_values(by='Correlacion')

         

         exotico = pd.Series(vectores_test[vectores_test['N_Pedido'] == resultados_correlacion_ordenado.iloc[0]['N_Pedido_test']].iloc[0])
         
         similar = resultados_correlacion[resultados_correlacion['N_Pedido_test'] == exotico['N_Pedido']].sort_values(by='Correlacion').iloc[-1]
         train_similar = pd.Series(vectores_train[similar['N_Pedido_train'] == vectores_train['N_Pedido']].iloc[0])
         
         diferencias_exotico, diferencias_similar = diferencias_componentes(exotico.iloc[ :-4], train_similar.iloc[ :-4])

         resultado = creacion_resultado_final(exotico, train_similar, diferencias_exotico, diferencias_similar)

         resultados_categorias = pd.concat([resultados_categorias, resultado], ignore_index=True)
         
         display(resultados_correlacion_ordenado)

Se asigna la etiqueta stuckliste

In [ ]:
df_stuckliste = pd.read_csv('TCU74A_Ibermatica.txt', delimiter=';', names = ['Serie','Pieza','Regla_Codigos','Fecha_hasta_validez_regla'])

resultados_categorias['Stuckliste'] = resultados_categorias['Id'].apply(lambda x: 'X' if any(df_stuckliste['Regla_Codigos'].str.contains(x)) else '')
resultados_categorias['Stuckliste'] = resultados_categorias['Id'].apply(lambda x: 'X' if any(df_stuckliste['Regla_Codigos'].str.contains(x)) else '')

Se asigna la etiqueta de chasis

In [ ]:
resultados_categorias['Chasis'] = resultados_categorias['Exotico'].map(data.set_index('N_Pedido')['Chasis'].apply(lambda x: 'X' if pd.notna(x) and x.strip() != '' else ''))

In [ ]:
ruta_guardado = 'Resultados\Resultados_Categorias.csv'
#nombre_resultado = 'Resultados_Categorias'
resultados_categorias.to_csv(ruta_guardado, index=False)